# Example Notebook to showcasing how we interact with JTReaders

In [1]:
# First change dir to JTR parent
import os
os.chdir('..')

### Bookkeeping of all existing readers: `readers.py`

In [2]:
import jtr.readers as readers

In [3]:
print("Existing models:\n%s" % ", ".join(readers.readers.keys()))

Existing models:
example_reader, modelf_reader, distmult_reader, complex_reader, transe_reader, fastqa_reader, cbow_xqa_reader, cbilstm_snli_reader, dam_snli_reader, esim_snli_reader, cbilstm_snli_streaming_reader


### Create a reader

In [4]:
%%script bash
bash data/GloVe/download.sh

glove.6B.50d.txt already exists! Doing nothing!


In [5]:
from jtr.io.embeddings.embeddings import load_embeddings
from jtr.util.vocab import Vocab

# we need a vocabulary (with embeddings for our fastqa_reader, but this is not always necessary)
embeddings = load_embeddings('data/GloVe/glove.6B.50d.txt', 'glove')
vocab = Vocab(emb=embeddings, init_from_embeddings=True)

# and a config
config = {"batch_size": 1, "repr_dim": 10, "repr_dim_input": embeddings.lookup.shape[1]}

In [6]:
# create example reader
from jtr.core import SharedResources

svac = SharedResources(vocab, config)
fastqa_reader = readers.readers["fastqa_reader"](svac)

### Setting up a reader from training data 

In [7]:
from jtr.data_structures import load_labelled_data
train_data = load_labelled_data('data/SQuAD/snippet.jtr.json')
# all parameters are initialized after this call
fastqa_reader.setup_from_data(train_data)

### Saving the reader

In [8]:
fastqa_reader.store("/tmp/fastqa_reader")

In [9]:
%%sh
ls /tmp/fastqa_reader/

checkpoint
model_module.data-00000-of-00001
model_module.index
model_module.meta
shared_resources


### Loading the reader

In [10]:
# we can simply load a setup reader
fastqa_reader.load("/tmp/fastqa_reader")

INFO:tensorflow:Restoring parameters from /tmp/fastqa_reader/model_module


In [11]:
# or setup a new reader from file 
import tensorflow as tf
from jtr.util.vocab import Vocab

# reset graph -> computation graph is gone
tf.reset_default_graph()
svac = SharedResources(None, None)
fastqa_reader = readers.readers["fastqa_reader"](svac)
fastqa_reader.load_and_setup("/tmp/fastqa_reader")

INFO:tensorflow:Restoring parameters from /tmp/fastqa_reader/model_module


### Applying the reader

In [12]:
from jtr.data_structures import load_labelled_data
train_data = load_labelled_data('data/SQuAD/snippet.jtr.json')

# take a list of inputs, e.g., from our training data
questions = [q for q, a in train_data]

In [13]:
for a in fastqa_reader(questions):
    print("{}, {}, {}".format(a.score, a.text, a.span))

0.0017765111988410354, simple, modern, (659, 673)
0.0016859363531693816, simple, modern, (659, 673)
0.001620858209207654, simple, modern, (659, 673)
0.0017491448670625687, simple, modern, (659, 673)
0.001751892501488328, simple, modern, (659, 673)
3.855477916658856e-05, and, (488, 491)
3.897287024301477e-05, and, (488, 491)
4.0846647607395425e-05, and, (488, 491)
3.922362157027237e-05, and, (488, 491)
4.22893172071781e-05, and, (488, 491)
0.00023499234521295875, center), Holy Cross House, as well as Columba Hall near the Grotto. The university through the Moreau Seminary has ties to theologian, (426, 559)
0.00010738338460214436, ,, (104, 105)
0.00011009203444700688, , which are in Rome). Its main seminary, Moreau Seminary, is located on the campus across St. Joseph lake from the Main Building. Old College, the oldest building on campus and located near the shore of St. Mary lake, houses undergraduate seminarians. Retired priests, (104, 371)
0.00011688958329614252, , which are in Rome).